<a href="https://colab.research.google.com/gist/yatochka-dev/0d27f9849c5f90646af325fd77fd9452/week4-philip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
from ultralytics import YOLO


In [ ]:
!unzip ./out_dataset.zip -d .


In [ ]:
yaml_content = """
path: out_dataset
train: images/train
val: images/val

names:
  0: B2
  1: B52
  2: C17
  3: C130
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

# Im too lazy to upload an actual file from my pc

Starting training, experiment #1: Model Size



In [ ]:
# 1. YOLOv8 Nano
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=640 name=model_n

In [ ]:
# 2. YOLOv8 Small
!yolo detect train data=data.yaml model=yolov8s.pt epochs=10 imgsz=640 name=model_s

In [ ]:
# 3. YOLOv8 Medium
!yolo detect train data=data.yaml model=yolov8m.pt epochs=10 imgsz=640 name=model_m

Continuing training, experiment #2: Learning Rate

In [ ]:
# 1. Learning rate - 0.001
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=640 lr0=0.001 name=lr_001

In [ ]:
# 2. Learning rate - 0.005
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=640 lr0=0.005 name=lr_005

In [ ]:
# 3. Learning rate - 0.01
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=640 lr0=0.01 name=lr_01

Final training round, experiment #3: Image Size

In [ ]:
# 1. Image Size: 416
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=416 name=imgsz_416

In [ ]:
# 2. Image Size: 640
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=640 name=imgsz_640

In [ ]:
# 3. Image Size: 1024
!yolo detect train data=data.yaml model=yolov8n.pt epochs=10 imgsz=1024 name=imgsz_1024

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Setup
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Your model size experiment names
experiments = [
    # ("model_n", "YOLOv8n"),
    # ("model_s", "YOLOv8s"),
    # ("model_m", "YOLOv8m"),
    # ("lr_001", "LR=0.001"),
        # ("lr_005", "LR=0.005"),
        # ("lr_01", "LR=0.01"),
    ("imgsz_416", "Img Size 416"),
        ("imgsz_640", "Img Size 640"),
        ("imgsz_1024", "Img Size 1024")
]

# Plot
for exp_name, label in experiments:
    path = Path(f"runs/detect/{exp_name}/results.csv")
    if not path.exists():
        print(f"Skipping {exp_name}: results.csv not found.")
        continue

    df = pd.read_csv(path)
    epochs = df["epoch"]
    mAP50 = df["metrics/mAP50(B)"]

    plt.plot(epochs, mAP50, marker='o', label=f"{label} (final: {mAP50.iloc[-1]:.3f})")

# Final touches
plt.title("Image size comparison – mAP@0.5 Over Epochs", fontsize=16)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("mAP@0.5", fontsize=12)
plt.xticks(epochs)
plt.legend(title="Model", loc="lower right")
plt.tight_layout()
plt.savefig("model_comparison_map50.png", dpi=300)
plt.show()


In [ ]:
import pandas as pd
from pathlib import Path

base_dir = Path("runs/detect")
experiments = sorted([d for d in base_dir.iterdir() if d.is_dir()])
summary = []

for exp in experiments:
    results_path = exp / "results.csv"
    if not results_path.exists():
        continue

    df = pd.read_csv(results_path)
    last_row = df.iloc[-1]  # get last epoch
    mAP50 = last_row.get("metrics/mAP50(B)", None)
    mAP50_95 = last_row.get("metrics/mAP50-95(B)", None)
    precision = last_row.get("metrics/precision(B)", None)
    recall = last_row.get("metrics/recall(B)", None)
    name = exp.name

    summary.append({
        "Experiment": name,
        "mAP@0.5": round(mAP50, 3) if pd.notnull(mAP50) else "N/A",
        "mAP@0.5:0.95": round(mAP50_95, 3) if pd.notnull(mAP50_95) else "N/A",
        "Precision": round(precision, 3) if pd.notnull(precision) else "N/A",
        "Recall": round(recall, 3) if pd.notnull(recall) else "N/A"
    })

# Display results
df_summary = pd.DataFrame(summary)
print("📊 Experiment Summary:")
print(df_summary.to_string(index=False))

# Optional: save to CSV
df_summary.to_csv("experiment_summary.csv", index=False)
